In [1]:
import findspark

In [2]:
findspark.init("/home/danial/spark-3.3.2-bin-hadoop3/")

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('recommender').getOrCreate()

23/04/13 11:38:48 WARN Utils: Your hostname, danial-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/04/13 11:38:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/13 11:38:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/13 11:38:50 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
path = '/home/danial/Desktop/myspark/Apache-Spark/Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Recommender_Systems/movielens_ratings.csv'

In [7]:
data = spark.read.csv(path, header=True, inferSchema=True)

In [8]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [9]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [10]:
from pyspark.ml.recommendation import ALS

In [11]:
from pyspark.ml.evaluation import RegressionEvaluator

In [12]:
training, testing = data.randomSplit([0.8, 0.2])

In [13]:
als = ALS(maxIter= 5, regParam=0.01, itemCol= 'movieId', userCol='userId', ratingCol='rating')

In [14]:
model = als.fit(training)

23/04/13 11:50:38 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/04/13 11:50:38 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
23/04/13 11:50:38 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [15]:
predictions = model.transform(testing)

In [16]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|      1|   1.0|    28|  0.9663938|
|      5|   2.0|    26| 0.02526009|
|      4|   1.0|    12|  0.8127196|
|      3|   2.0|    22|  1.6282926|
|      3|   1.0|     1|  1.3922048|
|      4|   2.0|    13|   1.782724|
|      6|   1.0|    20| 0.24720451|
|      4|   1.0|     5|  1.3330386|
|      2|   1.0|    19|  1.5031521|
|      4|   1.0|    19|  1.8913373|
|      1|   4.0|    15|  1.3782326|
|      6|   1.0|    15|  1.0345336|
|      2|   1.0|    17|  1.9824493|
|      0|   1.0|    23|  2.1761677|
|      4|   1.0|    23| 0.94977033|
|      2|   4.0|    10|  3.7171347|
|      4|   1.0|    24|0.024004161|
|      2|   4.0|    21|  2.5022135|
|      0|   1.0|    11|  1.6437361|
|      5|   1.0|    14|  3.3678727|
+-------+------+------+-----------+
only showing top 20 rows



In [17]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

In [18]:
rmse = evaluator.evaluate(predictions)

In [19]:
print (f"the RMSE is equal to {rmse}")

the RMSE is equal to 1.9813298445825118


In [20]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [21]:
# SO the RMSE of 1.98 is too high given the fact that the rating values are in the range of 1 to 5! 
# Which is because small dataset that we used!

In [26]:
single_user = testing.filter(testing['userId']==11).select(['movieId', 'userId'])
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      0|    11|
|     13|    11|
|     25|    11|
|     38|    11|
|     43|    11|
|     50|    11|
|     59|    11|
|     66|    11|
|     72|    11|
|     75|    11|
|     76|    11|
|     88|    11|
+-------+------+



In [27]:
recommendations = model.transform(single_user)

In [31]:
recommendations.orderBy('prediction', ascending = False).show()

+-------+------+------------+
|movieId|userId|  prediction|
+-------+------+------------+
|     76|    11|   5.5854764|
|     25|    11|    5.083869|
|     66|    11|    3.925778|
|     50|    11|   3.5083916|
|      0|    11|   1.6437361|
|     13|    11|   1.5627245|
|     75|    11|   1.1251544|
|     88|    11|  0.47486284|
|     59|    11|-0.043731105|
|     38|    11| -0.14139383|
|     72|    11| -0.66860205|
|     43|    11|  -1.7488561|
+-------+------+------------+

